In [1]:
import warnings
import os

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")

# Redirect warnings to null device
with open(os.devnull, "w") as devnull:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        warnings.warn("This is a sample warning.")  # Example warning that will be suppressed

        # Rest of your code

print("Print without warning")

Print without warning


# Import required Modules
* install scikit-multiflow to import drift detection algorithm
* Mount on google drive to load datasets.

In [2]:
!pip install -q scikit-multiflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.6/450.6 kB 33.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skmultiflow.drift_detection import ADWIN, DDM
from sklearn.neighbors import KNeighborsClassifier
# from warnings import simplefilter,filterwarnings
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from matplotlib.colors import ListedColormap
from multiprocessing.pool import ThreadPool
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from contextlib import suppress
from collections import Counter
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from random import shuffle
import seaborn as sns
from time import time
import pandas as pd
import numpy as np
import warnings
import scipy.io
import pickle
import sys
import gc
import os
import re
# filterwarnings('ignore')
# filterwarnings(action='once')
# warnings.filterwarnings("ignore", message="Skipping variable loading for optimizer 'Adam', because it has 13 variables whereas the saved optimizer has 1 variables.")
# warnings.filterwarnings("ignore", category=UserWarning, module="tensorflow")
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# # simplefilter('ignore', FutureWarning)
# with suppress(WARNING):

In [4]:
# from IPython.display import HTML

# HTML('''<script>
# var code_show_err = false;
# var code_toggle_err = function() {
#     var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
#     var stderr = Array.from(stderrNodes)
#     if (code_show_err){
#         stderr.forEach(ele => ele.style.display = 'block');
#     } else {
#         stderr.forEach(ele => ele.style.display = 'none');
#     }
#     code_show_err = !code_show_err
# }
# document.addEventListener('DOMContentLoaded', code_toggle_err);
# </script>
# To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data'
code_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/codes'
results_path = '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/results'

In [7]:
sys.path.insert(0,code_path)
from genetic_programming import SymbolicRegressor
from binirizer import CustomLabelBinirizer
from ensemble import Ensemble, Classifier
from oselm import OSELMClassifier,set_use_know
from DynamicFeatureSelection import dynamic_feature_selection

Print without warning
Print without warning
Print without warning
Print without warning


In [8]:
# ldseg=np.array(os.listdir(data_path))
# for filename in ldseg:
#   dst = re.sub('نسخة من ', '', filename)
#   src = os.path.join(data_path,filename)
#   dst = ospath.join(d, dst)
#   os.rename(src, dst)

In [9]:
# this files (genetic_programming, label_binirizer, pyoselm) must be uploaded to the google colab to be imported correctly.
# try:
#     del sys.modules['binirizer']
#     del sys.modules['genetic_programming']
#     del sys.modules['oselm']
# except:
#     pass

In [10]:
# from utils import load_pickle, save_pickle

# 1. Dataset Preprocessing :
* read the dataset using pnadas read_csv function.
* change the column names to numerical values
* apply label encoding on the vlaues of target variables (map each string value to numeric vlaue)

In [11]:
class Classifier:
    def __init__(self, clf, max_number_of_classes:int=2):
        """
        Wrapping sklearn classifiers
        clf: sklearn classifiers like (KNN, LogRegression, DecisionTree, etc...)
        max_number_of_classes: integer, number of unique values in the predicted variable.
        """
        self.clf = clf
        # decision profile contains the prediction probability values.
        self.decision_profile = None
        self.max_number_of_classes = max_number_of_classes


    # fit the classifier
    def fit(self, X_train, y_train, unselected_features=None):
        """
        Call the training function
        X_train: 2d array with shape num_of_samples x num_of_feautres.
        y_train: 1d array with shape (num_of_samples, ) contains the ground truth values.
        """
        # X_train = np.array(X_train) if not type(X_train).__module__ == np.__name__ else X_train
        # y_train = np.array(y_train) if not type(y_train).__module__ == np.__name__ else y_train
        # print("X_train : ",X_train.shape)
        # print("y_train : ",y_train.shape)
        if type(self.clf) == OSELMClassifier:
            self.clf.fit(X_train, y_train, unselected_features)
        else:
            # print("1234")
            self.clf.fit(X_train, y_train)
            # print(type(self.clf))

    def predict_proba(self, X):
        """
        predict the probability of belonging this `sample` to each class
        """
        # sometimes number of unique values in the predicted variable differ from one chunk to another,
        # so that we need to pad the results of probablity prediction to new size equal to `max_number_of_classes`

        pred = self.clf.predict_proba(X)
        return pred

    def build_decision_profile(self, sample):
        """
        add the predict_probability result to the `decision_profile` list
        sample: one example form the dataset
        """
        self.decision_profile = self.predict_proba(sample.reshape((1, -1)))[0].tolist()


class Ensemble:
    def __init__(self, classifiers, program, apply_model_replacement):

        """
        classfiers : list of Classifier objects
        program: result of genetic programming (SymbolicRegressor)
        """
        self.classifiers = classifiers
        self.program = program
        self.program_history = []
        self.fitted = False
        self.scores = {}
        self.apply_model_replacement = apply_model_replacement

    def fit(self, X_train, y_train, unselected_features=None):
        self.classifier_induction(self.classifiers, X_train, y_train, unselected_features=unselected_features)
        self.update_program(X_train, y_train)


    def classifier_induction(self, new_classifiers, X_train:np.array, y_train:np.array, unselected_features:list=None)->list:
        """
        new_classifiers: list of new classifiers to insert them into ensemble classifiers.
        X_train: training dataset .
        y_train: ground truth values.
        unselected_features: indices of unselected features at each chunk
        ----------------------------------------------------------------
        return new_classifiers after training.
        """
        # use classifier_induction_util for multiprocessing
        def classifier_induction_util(classifier):
            clf = Classifier(classifier, 2)
            clf.fit(X_train.copy(), y_train.copy(), unselected_features)
            return clf
        # train each new classifier in parallel
        trained_classifiers = ThreadPool(len(new_classifiers)).map(classifier_induction_util, new_classifiers)
        # add the trained classifiers to the ensemble classifiers.
        if self.apply_model_replacement:
          self.classifiers += trained_classifiers
        else:
          self.classifiers = trained_classifiers
        # return the trained classifiers (new classifiers after training)
        return trained_classifiers

    def model_replacement(self, criteria='best'):
        if criteria == 'best':
          pass
        elif criteria == 'time':
          self.classifiers = self.classifiers[3:]


    def global_support_degree(self, sample):
        for i,clf in enumerate(self.classifiers):
            if not isinstance(clf,Classifier):
              clf = Classifier(clf,2)
              self.classifiers[i] = clf
            clf.build_decision_profile(sample)
        profile = np.array([self.classifiers[i].decision_profile for i in range(len(self.classifiers))])
        return np.argmax(profile.sum(axis=0))

    def update_program(self, X, y):
        # change the fit flag to True.
        self.fitted = True
        profiles = np.array([self.classifiers[i].predict_proba(X) for i in range(len(self.classifiers))])
        self.program.fit(profiles, y)
        self.program_history.append(self.program)


    def predict(self, X_test):
        X_test = np.squeeze(X_test) if len(list(X_test.shape))>2 else X_test
        profiles = np.array([self.classifiers[i].predict_proba(X_test) for i in range(len(self.classifiers))])
        return self.program.predict(profiles)

    def evaluate(self, X_test, y_test, chunk_id=1):
        y_pred = self.predict(X_test)
        # accuracy_score, precision_score, recall_score, f1_score
        try:
          auc = roc_auc_score(y_test, y_pred)
        except:
          auc = 0.5
        self.scores[chunk_id] = {"accuracy": accuracy_score(y_test, y_pred),
                                 "precision": precision_score(y_test, y_pred),
                                 "recall": recall_score(y_test, y_pred),
                                 "f1-score": f1_score(y_test, y_pred),
                                 "auc": auc}
        print(self.scores)


# Componentes

In [12]:
filenames = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
             '7gradualDrift.csv', '7suddenDrift.csv']
filenames = list(map(lambda x: os.path.join(data_path, x), filenames))
filenames

['/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/kddcup99_csv.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/ISCX2012.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CSE-CIC2018.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/CICIDS2017.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7recurrentDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/blip.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/incrementalDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7gradualDrift.csv',
 '/content/drive/My Drive/Colab Notebooks/Muawiya/Genetic Programming Combiner with DFS/data/7suddenDrift.csv']

In [13]:
def prepare_data(csv_filename, target_column_name='class'):
    # read csv file
    df = pd.read_csv(csv_filename)
    df = df.iloc[:80000, :]
    column_names = df.columns.tolist()
    if target_column_name not in column_names:
        target_column_name = column_names[-1]
    # get unique value in target column
    unique_vlaues = sorted(df[target_column_name].unique().tolist())
    df[target_column_name] = df[target_column_name].apply(lambda x: 0 if x == unique_vlaues[0] else 1)
    df[target_column_name] = df[target_column_name].astype('int')
    # rename the column of the dataframe
    num_of_columns = len(column_names)
    df.columns = list(range(num_of_columns))
    return df

In [14]:
def train_and_test(model, X_train, y_train, X_test, y_test, unselected_features=None):
    model.fit(X_train, y_train, unselected_features)
    y_pred = model.predict(X_test)
    model.fit(X_test, y_test, unselected_features)
    return model, y_pred

In [15]:
def genetic_programming():
    return SymbolicRegressor(population_size=10,
            generations=5, stopping_criteria=0.85,
            p_crossover=0.7, p_subtree_mutation=0.1,
            p_hoist_mutation=0.05, p_point_mutation=0.1,
            max_samples=0.7, verbose=1,
            parsimony_coefficient=1e-4, random_state=42,
            function_set=['avg2', 'avg3', 'avg5',
                          'median3', 'median5', 'maximum2', 'maximum3', 'maximum5'],
            metric='f1-score')

In [16]:
def generate_oselm_models(number_of_hidden_neurons, apply_model_replacement=False):
    models= [OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             OSELMClassifier(number_of_hidden_neurons, 'relu', binarizer=CustomLabelBinirizer(), random_state=42),
             ]

    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

def generate_ml_models(number_of_hidden_neurons, apply_model_replacement=False):
    models = [
              KNeighborsClassifier(5),
              KNeighborsClassifier(5),
              # DecisionTreeClassifier(),
              LogisticRegression(),
              LogisticRegression(),
              GaussianNB(),
              GaussianNB(),
              GaussianNB(),
              ]
    ensemble = Ensemble(classifiers=models, program=genetic_programming(), apply_model_replacement=apply_model_replacement)
    return ensemble

In [17]:
def concept_drift_detection(drift_detection_obj, sample) -> bool:
    """
    Detect concept drift
    :param drift_detection_obj: sklearn drift detection object (ADWIN, DDM, )
    :param smaple : new instanece of data stream
    return True if concept drift was detected otherwise false
    """
    drift_detection_obj.add_element(sample)
    return drift_detection_obj.detected_change()

In [18]:
def feature_evolving(evolving_matrix):
    """
    evolving_matrix : list of random list
    """
    random_index = np.random.randint(0, len(evolving_matrix), 1)[0]
    return evolving_matrix[random_index]

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
def random_forest_feature_selection(X, y):
    """
    return best feature from X using random forest
    """
    sel = SelectFromModel(RandomForestClassifier(n_estimators = 20))
    sel.fit(X, y)
    return sel.get_support()

In [20]:
def main(datasets_paths, generate_model, train_size=25, transfer_learning=False, feature_selection=False,apply_model_replacement=False,chunk_number=0):
  """
  datasets_paths: list of paths (absolute path for each dataset)
  generate_model: function to generate machine learning model.
  train_size: number of samples to be used in training phase.
  transfer_learning: to determine using of transfer learning in the training phase.
  """
  # load the dataset and then process it
  datasets = {}
  for f_name in datasets_paths:
      d = prepare_data(f_name)
      datasets[f_name.split('/')[-1]] = d
  results = {}
  ########################################
  for key in tqdm(datasets.keys()):

      # convert dataset from dataframe to numpy array.
      data = datasets[key].values

      # split the data into features array and target array.
      X, Y = data[:, 0:-1], data[:, -1].astype('int')
      model = generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2,apply_model_replacement=apply_model_replacement) # generate_oselm_models

      # split the data into chunks (10 chunks)
      chunks_features = np.vsplit(X, 10)
      chunks_labels = np.split(Y, 10)

      results[key] = {}
      ################# train on each chunk ####################
      print("===================== dataset : {} ======================".format(key))
      chunk_number = 1
      for chunk_X, chunk_Y in tqdm(zip(chunks_features, chunks_labels)):
          if feature_selection:
            selected = random_forest_feature_selection(chunk_X, chunk_Y)
            unselected_feautres = np.where(selected != 1)[0]
          else:
            selected = list(range(0, X.shape[1]))
            unselected_feautres = None
          X_train, X_test, y_train, y_test = chunk_X[:train_size], chunk_X[train_size:], chunk_Y[:train_size], chunk_Y[train_size:]
          if transfer_learning:
            model, y_pred = train_and_test(model, X_train, y_train, X_test, y_test, unselected_feautres)
          else:
            _, y_pred = train_and_test(generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2,apply_model_replacement=apply_model_replacement), X_train, y_train, X_test, y_test, unselected_feautres)
          results[key][chunk_number] = {"y_true" : y_test, "y_pred": y_pred}
          chunk_number += 1
  return results

In [21]:
# oselm_result = main(filenames, generate_oselm_models, transfer_learning=False)

In [22]:
# oselm_result_transfer = main(filenames, generate_oselm_models, transfer_learning=True, feature_selection=False)

In [23]:
# ml_result = main(filenames, generate_ml_models, transfer_learning=False, feature_selection=False)

In [24]:
# oselm_result_transfer_gp = main(filenames, generate_oselm_models, transfer_learning=True, feature_selection=False)

In [25]:
# oselm_result_transfer_gp = main(filenames, generate_oselm_models, transfer_learning=True, feature_selection=True)

In [26]:
def E2SC4ID (X,
             y,
             sample_index:int,
             buffer:list,
             ensemble: Ensemble,
             drift_detection_obj,
             maxC=8,
             n=2000,
             train_size=0.5,
             drift=False,
             unselected_features=None,
             drift_location={}):
    """
    """
    y_pred = ensemble.global_support_degree(X)

    # if the sample is labeled then insert it into buffer
    if y is not None:
        buffer.append((X, y))
        actual_drift = concept_drift_detection(drift_detection_obj, int(y!=y_pred))
        if actual_drift and not drift:
          drift_location[sample_index] = 'drift'
        drift = drift or actual_drift
        if len(buffer) >= n:
            if drift:
                drift = False
                drift_detection_obj.reset()
                x_buffer, y_buffer = [], []
                for tup in buffer:
                    x_buffer.append(tup[0])
                    y_buffer.append(tup[1])
                ######################################################
                train_size = int(len(x_buffer)*train_size)
                X_train = x_buffer[:train_size]
                y_train = y_buffer[:train_size]
                X_valid = x_buffer[train_size:]
                y_valid = y_buffer[train_size:]
                ######################################################
                __sum = np.array(y_train).sum()
                if 0 ==  __sum or __sum == len(y_train):
                  y_train[0] = 0 if y_train[0] == 1 else 1
                new_models = ensemble.classifier_induction([
                                        model.clf for model in ensemble.classifiers],
                                        X_train,
                                        y_train,
                                        unselected_features)
                if len(ensemble.classifiers) > maxC:
                    ensemble.model_replacement('time')
                ######################################################
                ensemble.update_program(X_valid, y_valid)

            else:
                buffer.clear()
        return ensemble, buffer, drift, drift_location

In [27]:
def E2SC4ID_STREAM(ensemble, stremdata, y, unselected_features, drift_location, chunk_number):
    """
    """
    if not ensemble.fitted:
      y[0] = 0 if y[0] == 1 else 1
      ensemble.fit(stremdata[:200], y[:200])

    drift_detection_obj = DDM()
    drift = False
    buffer = []
    for i in tqdm(range(200, len(stremdata))):
        X, y_true = stremdata[i], y[i]
        ensemble, buffer, drift, drift_location = E2SC4ID (X,
                                                           y_true,
                                                           sample_index=(i +(chunk_number * 10000)),
                                                           buffer=buffer,
                                                           ensemble=ensemble,
                                                           drift_detection_obj=drift_detection_obj,
                                                           maxC=8,
                                                           n=2000,
                                                           train_size=0.7,
                                                           drift=drift,
                                                           unselected_features=unselected_features,
                                                           drift_location=drift_location)
    return ensemble, drift_location

In [28]:
def save_pickle(obj, file_name):
  with open(file_name, 'wb') as f:
    pickle.dump(obj, f)
def load_pickle(file_name):
  with open(file_name, 'rb') as f:
    d = pickle.load(f)
  return d

In [29]:
def generate_new_samples(buffer, y_values, n=500, y_col='label'):
    if not y_col in buffer.columns.tolist():
      y_col = buffer.columns.tolist()[-1]
    if y_values.sum() == 0:
       return buffer[buffer[y_col] == 1].sample(n, random_state=41)[:, :-1].values, np.array([1] * n)
    else:
      return buffer[buffer[y_col] == 0].sample(n,random_state=41)[:, :-1].values, np.array([0] * n)

In [30]:
def main(f_name, generate_model, train_size=3000,apply_model_replacement=False, transfer_learning=False, feature_selection="random_forest", result_save_path="",ChunkNumber=0):
  """
  datasets_paths: list of paths (absolute path for each dataset)
  generate_model: function to generate machine learning model.
  train_size: number of samples to be used in training phase.
  transfer_learning: to determine using of transfer learning in the training phase.
  """
  # load the dataset and then process it
  datasets = {}

  d = prepare_data(f_name)
  d = d.sample(frac=1, random_state=42)
  buffer = d.sample(n=5000)
  d.reset_index(inplace=True)
  d.replace([np.inf], 0, inplace=True)
  datasets[f_name.split('/')[-1]] = d
  results = {}
  drift_locations_in_all_dataset = {}
  ensemble = None ####
  ########################################
  for key in tqdm(datasets.keys()):
      drift_location = {}
      results[key] = {'model_result': []}
      # convert dataset from dataframe to numpy array.
      data = datasets[key].values
      # split the data into features array and target array.
      X, Y = data[:, 0:-1], data[:, -1].astype('int')
      if not os.path.exists("{}_evolving_matrix.pkl".format(key)):
        a2 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 6).tolist()
        a3 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 5).tolist()
        a4 = np.random.randint(low=0, high=X.shape[1], size = X.shape[1] // 4).tolist()
        evolving_matrix = [a2, a3, a4]
        save_pickle(evolving_matrix, "{}_evolving_matrix.pkl".format(key))
      else:
        evolving_matrix = load_pickle("{}_evolving_matrix.pkl".format(key))
      ensemble = generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2, apply_model_replacement=apply_model_replacement)

      # split the data into chunks (10 chunks)
      chunks_features = np.array_split(X, 10)
      chunks_labels = np.array_split(Y, 10)

      result_save_path_data = os.path.join(result_save_path, key)
      ################# train on each chunk ####################
      print("===================== dataset : {} ======================".format(key))
      chunk_number = 1
      if ChunkNumber>0:
        ensemble = load_pickle(os.path.join(result_save_path_data, "{}_ensemble.pkl".format(key)))
        results = load_pickle(os.path.join(result_save_path_data, "{}_results.pkl".format(key)))
        drift_locations_in_all_dataset = load_pickle(os.path.join(result_save_path_data, "{}_drift_locations_in_all_dataset.pkl".format(key)))
      for CN,chunk_X, chunk_Y in tqdm(zip([*range(len(chunks_labels))],chunks_features, chunks_labels)):
          if ChunkNumber > CN:
            print("We Skip Chunk Number : {}".format(CN))
            continue
          try:
            chunk_X, chunk_Y = SMOTE().fit_resample(chunk_X, chunk_Y)
          except:
            if chunk_Y.sum() in [0, 1]:
              new_samples, new_labels = generate_new_samples(buffer, chunk_Y)
              chunk_X = np.concatenate((chunk_X, new_samples))
              chunk_Y = np.concatenate((chunk_Y, new_labels))
          gc.collect()
          unselected_feautres = None
          selected = None
          if feature_selection[0] == "feature_evolving":
            unselected_feautres = feature_evolving(evolving_matrix=evolving_matrix)
            chunk_X = np.delete(chunk_X, unselected_feautres, 1)
            if feature_selection[1] == "random_forest":
              selected = random_forest_feature_selection(chunk_X, chunk_Y)
              unselected_feautres = np.where(selected != 1)[0]
            elif feature_selection[1] == "DFS_feature_selection":
              selected = dynamic_feature_selection(chunk_X, chunk_Y)
              unselected_feautres = np.where(selected != 1)[0]
          else:
            if feature_selection[1] == "random_forest":
              selected = random_forest_feature_selection(chunk_X, chunk_Y)
              unselected_feautres = np.where(selected != 1)[0]
            elif feature_selection[1] == "DFS_feature_selection":
              selected = dynamic_feature_selection(chunk_X, chunk_Y)
              unselected_feautres = np.where(selected != 1)[0]

          X_train, X_test, y_train, y_test = chunk_X[:train_size], chunk_X[train_size:], chunk_Y[:train_size], chunk_Y[train_size:]
          if transfer_learning:
            temp = np.squeeze(X_train[:, selected]) if len(list(X_train[:, selected].shape))>2 else X_train[:, selected]
            ensemble, drift_location = E2SC4ID_STREAM(ensemble, temp, y_train, None, drift_location, chunk_number)
            temp = np.squeeze(X_test[:, selected]) if len(list(X_test[:, selected].shape))>2 else X_test[:, selected]
            ensemble.evaluate(temp, y_test, chunk_number)
          else:
            ensemble, drift_location = E2SC4ID_STREAM(generate_model(number_of_hidden_neurons=X.shape[1]*3 // 2,apply_model_replacement=apply_model_replacement),
                                                      X_train, y_train, unselected_feautres, drift_location, chunk_number)
            ensemble.evaluate(X_test, y_test, chunk_number)

          temp = np.squeeze(X_test[:, selected]) if len(list(X_test[:, selected].shape))>2 else X_test[:, selected]
          y_pre = ensemble.predict(temp)
          results[key][chunk_number] = {"y_true" : y_test, "y_pred": y_pre}
          results[key]['model_result'].append(ensemble.scores)
          if transfer_learning:
             ensemble.fit(temp, y_test, None)
          chunk_number += 1
          drift_locations_in_all_dataset[key] = drift_location
          if not os.path.exists(result_save_path_data):
            os.mkdir(result_save_path_data)
          save_pickle(ensemble, os.path.join(result_save_path_data, "{}_ensemble.pkl".format(key)))
          save_pickle(results, os.path.join(result_save_path_data, "{}_results.pkl".format(key)))
          save_pickle(drift_locations_in_all_dataset, os.path.join(result_save_path_data, "{}_drift_locations_in_all_dataset.pkl".format(key)))
  return ensemble, results, drift_locations_in_all_dataset

# with feature evolving and random_forest

In [31]:
# qq = ['kddcup99_csv.csv','ISCX2012.csv','CSE-CIC2018.csv','CICIDS2017.csv','7recurrentDrift.csv', 'blip.csv', 'incrementalDrift.csv',
#              '7gradualDrift.csv', '7suddenDrift.csv']

In [32]:
# # is done
# use_know = False
# set_use_know(use_know)
# path = os.path.join(results_path,'gp_combiner_with_model_replacement_evolving_random_forest')
# os.makedirs(path, exist_ok=True)
# for i,f_name in enumerate(filenames):
#     oselm_model_no_transfer, oselm_result_no_transfer, drift_locations_in_all_dataset_ = main(f_name, generate_oselm_models, apply_model_replacement=True,
#                                                                                               transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [33]:
# # is done
# use_know = False
# set_use_know(use_know)
# path = os.path.join(results_path,'faoselm_gp_combiner_with_model_replacement_evolving_random_forest')
# os.makedirs(path, exist_ok=True)
# for i,f_name in enumerate(filenames):
#   faoselm_result_transfer, faoselm_result_transfer_result, drift_locations_in_all_dataset_1 = main(f_name, generate_oselm_models, apply_model_replacement=True,
#                                                                                                    transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
# ChunkNumber=0
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[0]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path,ChunkNumber=9)

  0%|          | 0/1 [00:00<?, ?it/s]

===================== dataset : kddcup99_csv.csv ======================


0it [00:00, ?it/s]

We Skip Chunk Number : 0
We Skip Chunk Number : 1
We Skip Chunk Number : 2
We Skip Chunk Number : 3
We Skip Chunk Number : 4
We Skip Chunk Number : 5
We Skip Chunk Number : 6
We Skip Chunk Number : 7
We Skip Chunk Number : 8


  0%|          | 0/2800 [00:00<?, ?it/s]

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[1]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[2]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[3]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[4]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[5]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[6]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[7]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = True
set_use_know(use_know)
path = os.path.join(results_path,'kpfaoselm_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[8]]:
  kpfaoselm_result_transfer, kpfaoselm_result_transfer_result, drift_locations_in_all_dataset_2 = main(f_name, generate_oselm_models, apply_model_replacement=True,
                                                                                                   transfer_learning=True, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[0]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[1]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[2]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[3]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[4]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[5]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[6]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[7]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)

In [ ]:
# is not running
use_know = False
set_use_know(use_know)
path = os.path.join(results_path,'ml_gp_combiner_with_model_replacement_evolving_random_forest')
os.makedirs(path, exist_ok=True)
for f_name in [filenames[8]]:
  ml_result_transfer, ml_result_transfer_result, drift_locations_in_all_dataset_4 = main(f_name, generate_ml_models, apply_model_replacement=True,
                                                                                                   transfer_learning=False, feature_selection=["feature_evolving",'random_forest'], result_save_path=path)